<a href="https://colab.research.google.com/github/dlkt101101/AMATH445/blob/main/AMATH445_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMATH 445
## Assignment 2
Prepared by: Darren Alexander Lam Kin Teng (20977843)

# Question 1

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import math
import torch
import torch.nn as nn
from torch.nn import ReLU, Tanh
import torch.optim as optim
from sklearn.model_selection import KFold
from itertools import product
from sklearn.metrics import classification_report, mean_squared_error

In [2]:
with open('/content/A2Q1_data_regression.pkl', 'rb') as file:
  data_regression = pickle.load(file)

with open('/content/A2Q1_data.pkl', 'rb') as file:
  data_classification = pickle.load(file)

In [3]:
print(data_regression.keys(), data_classification.keys())

dict_keys(['X_train', 'X_test', 'y_train', 'y_test']) dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])


In [4]:
# regression
X_train_regression = data_regression['X_train']
y_train_regression = data_regression['y_train']
X_test_regression = data_regression['X_test']
y_test_regression = data_regression['y_test']

# classification
X_train_classification = data_classification['X_train']
y_train_classification = data_classification['y_train']
X_test_classification = data_classification['X_test']
y_test_classification = data_classification['y_test']

In [5]:
print('Checking Regression dataset split:')
print(X_train_regression.shape[0]/(X_train_regression.shape[0]+X_test_regression.shape[0]))

print('Checking Classification dataset split:')
print(X_train_classification.shape[0]/(X_train_classification.shape[0]+X_test_classification.shape[0]))

Checking Regression dataset split:
0.847457627118644
Checking Classification dataset split:
0.847457627118644


## Model Development

There will only be one node for both cases of binary classification and regression.

The input layer will contain the same number of nodes as our explanatory variables.

The number of neurons per hidden layer is based on a rule of thumb:
$Neurons = \frac{2}{3}(\text{neurons in input layer})+1$
(https://www.heatonresearch.com/2017/06/01/hidden-layers.html)

In [6]:
bool_cols_class = X_train_classification.select_dtypes(include='bool').columns
X_train_classification[bool_cols_class] = X_train_classification[bool_cols_class].astype(int)
X_test_classification[bool_cols_class] = X_test_classification[bool_cols_class].astype(int)

bool_cols_regress = X_train_regression.select_dtypes(include='bool').columns
X_train_regression[bool_cols_regress] = X_train_regression[bool_cols_regress].astype(int)
X_test_regression[bool_cols_regress] = X_test_regression[bool_cols_regress].astype(int)

In [7]:
input_layer_regression = X_train_regression.shape[1]
input_layer_classification = X_train_classification.shape[1]
# N_neurons_regression = math.floor(2/3 * input_layer_regression) + 1
# N_neurons_classification = math.floor(2/3 * input_layer_classification) + 1
N_neurons_regression = 10
N_neurons_classification = 10
output_layer = 1

EPOCHS = 500

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, problem="classification", activation_function='relu'):
        super().__init__()
        self.problem = problem
        self.activation_function_type = activation_function

        if problem == "classification":
          INPUT_SIZE, N_NEURONS = input_layer_classification, N_neurons_classification
        elif problem == "regression":
          INPUT_SIZE, N_NEURONS = input_layer_regression, N_neurons_regression
        else:
          print("Invalid problem. Problem must be 'regression' or 'classification'.")

        # building the neural network
        self.layers = nn.Sequential(
            nn.Linear(INPUT_SIZE, N_NEURONS),
            nn.BatchNorm1d(N_NEURONS),
            Tanh() if self.activation_function_type in ['mixed', 'tanh'] else ReLU(),
            nn.Linear(N_NEURONS, N_NEURONS),
            nn.BatchNorm1d(N_NEURONS),
            ReLU() if self.activation_function_type in ['mixed', 'relu'] else Tanh(),
            nn.Linear(N_NEURONS, N_NEURONS),
            nn.BatchNorm1d(N_NEURONS),
            ReLU() if self.activation_function_type in ['mixed', 'tanh'] else Tanh(),
            nn.Linear(N_NEURONS, output_layer))

    def forward(self, x):
      logits = self.layers(x)
      if self.problem == "classification":
        return torch.sigmoid(logits)
      return logits

In [9]:
kf = KFold(n_splits = 5, random_state=123, shuffle=True)

In [10]:
def train_evaluate_model_cv(problem_type='classification', learning_rate = [0.0001, 0.001],
                            momentum = [0.0, 0.9], activation_function = ['relu', 'tanh', 'mixed']):

  X_raw = X_train_classification if problem_type == 'classification' else X_train_regression
  y_raw = y_train_classification if problem_type == 'classification' else y_train_regression

  X = torch.tensor(X_raw.values, dtype=torch.float32)
  y = torch.tensor(y_raw.values, dtype=torch.float32)

  parameter_grid = {'lr' : learning_rate,
                    'mom' : momentum,
                    'act' : activation_function}

  combinations = list(product(parameter_grid['lr'], parameter_grid['mom'], parameter_grid['act']))

  # store the best hyper parameters
  best_loss = math.inf
  best_parameters = {}

  for lr, mom, act in combinations:
    losses = []
    for i, (train_index, val_index) in enumerate(kf.split(X)):
      X_train, X_val = X[train_index], X[val_index]
      y_train, y_val = y[train_index], y[val_index]

      model = NeuralNetwork(problem=problem_type, activation_function=act)
      optimizer = optim.Adam(model.parameters(), lr=lr, betas=(mom, 0.999))

      # specify the criterion for classification and regression
      if problem_type == 'regression':
        criterion = nn.MSELoss()
      else:
        criterion = nn.BCELoss()

      model.train()
      for epoch in range(EPOCHS):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train.view_as(outputs))
        loss.backward()
        optimizer.step()

      # model evaluation
      model.eval()
      with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val.view_as(val_outputs))
        losses.append(val_loss.item())

    avg_fold_loss = np.mean(losses)

    if avg_fold_loss < best_loss:
      best_loss = avg_fold_loss
      best_parameters = {'lr' : lr,
                         'mom' : mom,
                         'act' : act}

  return (best_loss, best_parameters)


In [11]:
class_params = train_evaluate_model_cv(problem_type='classification')

In [12]:
regress_params = train_evaluate_model_cv(problem_type='regression')

Below we get the results from our deep learning models for both regression and classification along with their best parameters.

In [13]:
class_params

(np.float64(0.6477210402488709), {'lr': 0.0001, 'mom': 0.0, 'act': 'tanh'})

In [14]:
regress_params

(np.float64(5160.131640625), {'lr': 0.001, 'mom': 0.0, 'act': 'mixed'})

Final Model

In [15]:
best_lr_class = class_params[1]['lr']
best_mom_class = class_params[1]['mom']
best_act_class = class_params[1]['act']

final_model_class = NeuralNetwork(problem="classification", activation_function=best_act_class)
optimizer_class = optim.Adam(final_model_class.parameters(), lr=best_lr_class, betas=(best_mom_class, 0.999))
criterion_class = nn.BCELoss()

In [16]:
best_lr_regress = regress_params[1]['lr']
best_mom_regress = regress_params[1]['mom']
best_act_regress = regress_params[1]['act']

final_model_regress = NeuralNetwork(problem="regression", activation_function=best_act_regress)
optimizer_regress = optim.Adam(final_model_regress.parameters(), lr=best_lr_regress, betas=(best_mom_regress, 0.999))
criterion_regress = nn.MSELoss()

In [17]:
X_train_c = torch.tensor(X_train_classification.values, dtype=torch.float32)
y_train_c = torch.tensor(y_train_classification.values, dtype=torch.float32).view(-1, 1)

X_test_c = torch.tensor(X_test_classification.values, dtype=torch.float32)
y_test_c = torch.tensor(y_test_classification.values, dtype=torch.float32).view(-1, 1)

X_train_r = torch.tensor(X_train_regression.values, dtype=torch.float32)
y_train_r = torch.tensor(y_train_regression.values, dtype=torch.float32).view(-1, 1)

X_test_r = torch.tensor(X_test_regression.values, dtype=torch.float32)
y_test_r = torch.tensor(y_test_regression.values, dtype=torch.float32).view(-1, 1)

Training the models

In [18]:
# training classification model
final_model_class.train()
for epoch in range(EPOCHS):
  optimizer_class.zero_grad()
  outputs_class = final_model_class(X_train_c)
  loss_class = criterion_class(outputs_class, y_train_c)
  loss_class.backward()
  optimizer_class.step()

In [20]:
# training regression model
final_model_regress.train()
for epoch in range(EPOCHS):
  optimizer_regress.zero_grad()
  outputs_regress = final_model_regress(X_train_r)
  loss_regress = criterion_regress(outputs_regress, y_train_r)
  loss_regress.backward()
  optimizer_regress.step()

Predicting with the models

In [21]:
final_model_class.eval()
with torch.no_grad():
    # Get raw probabilities
    raw_probs = final_model_class(X_test_c)
    preds_class = (raw_probs > 0.5).float()

y_true_class = y_test_c.numpy()
y_pred_class = preds_class.numpy()

In [22]:
# regression prediction
final_model_regress.eval()
with torch.no_grad():
    preds_regress = final_model_regress(X_test_r)

y_true_regress = y_test_r.numpy()
y_pred_regress = preds_regress.numpy()

Evaluating final models

In [23]:
class_report = classification_report(y_true_class, y_pred_class)
print(class_report)

              precision    recall  f1-score   support

         0.0       0.71      0.67      0.69        36
         1.0       0.40      0.44      0.42        18

    accuracy                           0.59        54
   macro avg       0.55      0.56      0.55        54
weighted avg       0.60      0.59      0.60        54



In [25]:
mse = mean_squared_error(y_true_regress, y_pred_regress)
print('The regression NN has a MSE: {}'.format(round(mse, 6)))

The regression NN has a MSE: 5369.636719
